In [1]:
# import library

import ctypes
import gc

In [2]:
# create a function for reference count

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [18]:
# create a function to dig into the gabage collector - is that object exists in the garbage collector

def obj_by_id(obj_id): # object id which we pass in
    for obj in gc.get_objects(): # look into every objects in the collector
        if id(obj) == obj_id: # compare memory addresses by usning the id
            return 'Object exists!'
    return "Not found!" # if don't return not found

In [4]:
# create classes that are going to create a circular reference

class A:
    def __init__(self): # a instuctor
        self.b = B(self) # another class called B - B(self) self here means the instance A
        print('A: self: {0}, b: {1}'.format(hex(id(self)), hex(id(self.b)))) # print out momory address

In [5]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

In [6]:
# here are are going to disble the garbage collector - to see what is going on

gc.disable()

In [7]:
my_var = A()

B: self: 0x105a08050, a: 0x105a0fb90
A: self: 0x105a0fb90, b: 0x105a08050


In [8]:
# let's look at the id of my_var

hex(id(my_var)) # memory of my_var should be same exactly as A

'0x105a0fb90'

In [9]:
# look at in another way - print momry address as well

print(hex(id(my_var.b)))

print(hex(id(my_var.b.a)))

0x105a08050
0x105a0fb90


In [11]:
# now look at the reference count

a_id = id(my_var)
b_id = id(my_var.b)

In [13]:
# check

print(a_id)
print(b_id)

print(hex(a_id))
print(hex(b_id))

4389403536
4389371984
0x105a0fb90
0x105a08050


In [14]:
# call a method ref_count

ref_count(a_id)

2

In [15]:
ref_count(b_id) # bcs only a pointing to b

1

In [19]:
# now we are going to look in the garbage

obj_by_id(a_id) # pass in the id of a

'Object exists!'

In [20]:
obj_by_id(b_id)

'Object exists!'

In [21]:
# here we are going to set my_var to None - destroy that reference between my_var and a object

my_var = None

In [22]:
# let's look

ref_count(a_id)

1

In [23]:
ref_count(b_id)

1

In [26]:
# we got rid of my_var but a is still pointing to b and b is still pointing to a bcs it is a circular reference
# therefore is not going to to help us out there bcs the reference count is non-zero
# that's where the garbage collector comes in!

In [27]:
# we can make sure that by checking if those objects are indeed being tracked by the garbage collector

obj_by_id(a_id)

'Object exists!'

In [28]:
obj_by_id(b_id)

'Object exists!'

In [29]:
# They have the reference count and the garbage collector is tracking them, but it has NOT clean them up.
# Why? It is bcs we stopped the garbage collector, remember? haha

In [30]:
# now are are going to run the garbage collejctor manually

gc.collect()

1680

In [32]:
# look at those objects again after we run the garbage colloctor

obj_by_id(a_id)

'Not found!'

In [33]:
obj_by_id(b_id)

'Not found!'

In [34]:
# They are GONE!

In [35]:
ref_count(a_id)

2

In [36]:
ref_count(b_id)

2

In [37]:
ref_count(b_id)

2

In [38]:
ref_count(a_id)

2

In [ ]:
# We are not see the object BUT still getting a reference count and it is seems to change over time
# Why? This is why we usually don't use memory address and we don't work with memory addresses in Python unless we are trying to debug stuff or trying to do our own memory clean up
# We gotto be really careful using memory addresses